In [1]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline 
import nltk
from nltk.tokenize import MWETokenizer
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn import metrics
import numpy as np
from nltk.stem import WordNetLemmatizer
import re
from nltk.tokenize import RegexpTokenizer

In [2]:
lab_data = pd.read_csv('labeled_data.csv')

In [3]:
def remove_extra_characters(raw_text):
    processed_text = re.sub('\\n','', raw_text)
    processed_text = re.sub('\\r','', processed_text)
    processed_text = re.sub("\\'", "\'",processed_text)
    return processed_text

In [4]:
lab_data['text'] = lab_data.apply(lambda row: remove_extra_characters(row['text'].strip()), axis=1)

In [5]:
stopwords = []
with open('./stopwords_en.txt') as f:
    stopwords = f.read().splitlines()
stopwords = set(stopwords)

In [6]:
def lemmatization(token_list):
    lemmatizer = WordNetLemmatizer()
    lem_token = []
    for each in token_list :
#         print(each ,":", lemmatizer.lemmatize(each)) 
        lem_token.append(lemmatizer.lemmatize(each))
    return lem_token

In [7]:
tokenizer = RegexpTokenizer("\w+(?:[']\w+)?")

In [8]:
def token(raw_data):
 #   raw_data1 = raw_data.lower()
    tokenised = tokenizer.tokenize(raw_data)
#     tokenised = nltk.tokenize.word_tokenize(raw_data1)
    lem_token = lemmatization(tokenised)
#     stopwords_tokens = [w for w in tokenised if not w in stopwords]
    processed_data = ' '.join(lem_token)
        
    return(processed_data)

In [9]:
lab_data['text'] = lab_data.apply(lambda row: token(row['text'].strip()), axis=1)

# Keras


In [10]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

Using TensorFlow backend.


In [11]:
from nltk.tokenize import word_tokenize

all_words = []
for sent in lab_data['text']:
    tokenize_word = word_tokenize(sent)
    for word in tokenize_word:
        all_words.append(word)

In [12]:
unique_words = set(all_words)
print(len(unique_words))

74426


In [15]:
vocab_length = 7500

In [17]:
embedded_sentences = [one_hot(sent, vocab_length) for sent in lab_data['text']]
#print(embedded_sentences)

In [18]:
word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(lab_data['text'], key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

In [19]:
padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')
print(padded_sentences)

[[2305 3461  497 ...    0    0    0]
 [2673 2904 7484 ...    0    0    0]
 [3167 1736 1959 ...    0    0    0]
 ...
 [2434 2357 5971 ...    0    0    0]
 [5787 3167 3481 ...    0    0    0]
 [3573 6634 5312 ...    0    0    0]]


In [56]:
model = Sequential()
model.add(Embedding(vocab_length, 20, input_length=length_long_sentence))
model.add(Flatten())
model.add(Dense(6, activation='softmax'))

In [57]:
model.compile(optimizer='adam',loss = 'categorical_crossentropy', metrics=['acc'])
#print(model.summary())

In [44]:
#model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
#print(model.summary())

In [ ]:
#test_model.fit(x=train_word, y=train_y, epochs=1, batch_size=70)

In [47]:
sentiments =  lab_data['label'].tolist()

In [48]:
sentiments = array(sentiments)
print(type(sentiments))

<class 'numpy.ndarray'>


In [50]:
from keras.utils import to_categorical
sentiments = to_categorical(sentiments)

In [ ]:
model.fit(padded_sentences, sentiments, epochs=50, verbose=1)

Epoch 1/50
50000/50000 [==============================] - 14s 283us/step - loss: 0.4042 - acc: 0.8671
Epoch 2/50
50000/50000 [==============================] - 14s 286us/step - loss: 0.3236 - acc: 0.9009
Epoch 3/50
50000/50000 [==============================] - 16s 313us/step - loss: 0.2564 - acc: 0.9268
Epoch 4/50
50000/50000 [==============================] - 17s 335us/step - loss: 0.2011 - acc: 0.9478
Epoch 5/50
50000/50000 [==============================] - 16s 312us/step - loss: 0.1561 - acc: 0.9634
Epoch 6/50
50000/50000 [==============================] - 15s 306us/step - loss: 0.1201 - acc: 0.9751
Epoch 7/50
50000/50000 [==============================] - 16s 310us/step - loss: 0.0906 - acc: 0.9839
Epoch 8/50
50000/50000 [==============================] - 16s 325us/step - loss: 0.0671 - acc: 0.9895
Epoch 9/50
50000/50000 [==============================] - 16s 315us/step - loss: 0.0495 - acc: 0.9928
Epoch 10/50
50000/50000 [==============================] - 16s 319us/step - loss: 

In [59]:
loss, accuracy = model.evaluate(padded_sentences, sentiments, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 88.792002


In [60]:
#loss, accuracy = model.evaluate(padded_sentences, sentiments, verbose=0)
#print('Accuracy: %f' % (accuracy*100))